In [119]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import zlib  
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
import tensorflow as tf
from skimage import io
from PIL import Image
import random

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [120]:
url = 'http://ufldl.stanford.edu/housenumbers/'
last_percent_reported = None
data_root = '.' # Change me to store data elsewhere

def download_progress_hook(count, blockSize, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 5% change in download progress.
  """
  global last_percent_reported
  percent = int(count * blockSize * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent
        
def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  dest_filename = os.path.join(data_root, filename)
  if force or not os.path.exists(dest_filename):
    print('Attempting to download:', filename) 
    filename, _ = urlretrieve(url + filename, dest_filename, reporthook=download_progress_hook)
    print('\nDownload Complete!')
  statinfo = os.stat(dest_filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', dest_filename)
  else:
    raise Exception(
      'Failed to verify ' + dest_filename + '. Can you get to it with a browser?')
  return dest_filename

train_filename = maybe_download('train.tar.gz', 404141560)
test_filename = maybe_download('test.tar.gz', 276555967)

Found and verified ./train.tar.gz
Found and verified ./test.tar.gz


In [121]:
num_classes = 10
np.random.seed(133)
data_folders = []

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0][2:]  # remove .tar.gz
  print(root)
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall(data_root)
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  
  return root

test_folders = maybe_extract(test_filename)
train_folders = maybe_extract(train_filename)
print("Done")


test
test already present - Skipping extraction of ./test.tar.gz.
train
train already present - Skipping extraction of ./train.tar.gz.
Done


In [122]:
def remove_files(folder):
    files = os.listdir(folder)[:-2]
    if 'digitStruct.mat' in files:
        os.remove(os.path.join(folder, "digitStruct.mat"))
    if 'see_bboxes.m' in files:
        os.remove(os.path.join(folder, 'see_bboxes.m'))    
    random.shuffle(files)

remove_files(train_folders)
remove_files(test_folders)


In [123]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load_letter(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size, 3),
                         dtype=np.float32)
  num_images = 0
  for image in image_files:
    image_file = os.path.join(folder, image)

    try:
      img = Image.open(image_file) # image extension *.png,*.jpg
      img = img.resize((image_size, image_size), Image.ANTIALIAS)
      img.save(image_file) # format may what u want ,*.png,*jpg,*.gif
      image_data = (mpimg.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      image_data = mpimg.imread(image_file).astype(float) 
      if image_data.shape != (image_size, image_size, 3):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[num_images, :, :] = image_data
      num_images = num_images + 1
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset
        
def maybe_pickle(folder, min_num_images_per_class, force=False):
  set_filename = folder + '.pickle'
  dataset_names = set_filename
  if os.path.exists(set_filename) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping pickling.' % set_filename)
  else:
    print('Pickling %s.' % set_filename)
    dataset = load_letter(folder, min_num_images_per_class)
    try:
      with open(set_filename, 'wb') as f:
        pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
      print('Unable to save data to', set_filename, ':', e)

  return dataset_names

# train_datasets = maybe_pickle(train_folders, 30000)
train_datasets = maybe_pickle(train_folders, 1)
# test_datasets = maybe_pickle(test_folders, 5000)
test_datasets = maybe_pickle(test_folders, 1)

Pickling train.pickle.
Full dataset tensor: (33402, 28, 28, 3)
Mean: 0.437628
Standard deviation: 0.205923
Pickling test.pickle.
Full dataset tensor: (13068, 28, 28, 3)
Mean: 0.431104
Standard deviation: 0.234512


In [124]:
def make_arrays(nb_rows, img_size):
  if nb_rows:
    dataset = np.ndarray((nb_rows, img_size, img_size,3), dtype=np.float32)
#     dataset = np.ndarray((nb_rows, img_size, img_size), dtype=np.float32)
    labels = np.ndarray(nb_rows, dtype=np.int32)
  else:
    dataset, labels = None, None
  return dataset, labels

def define_labels(label_files, dataset, start, end):
    for i, label in enumerate(label_files[start:end]):
        dataset[i] = long(label.split('.')[0][-1:])
#         dataset[i] = long(label.split('.')[0])
    print(dataset[end-1])
    return dataset

def merge_datasets(pickle_file, train_size, valid_size=0):
    num_classes = 10
    valid_dataset, valid_labels = make_arrays(valid_size, image_size)
    train_dataset, train_labels = make_arrays(train_size, image_size)

    start_v, start_t = 0, 0
    end_v, end_t = valid_size, train_size
    end_l = valid_size+train_size
    label_files = os.listdir(pickle_file[:-7])
    try:
      with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)
        if valid_dataset is not None:
          valid_letter = letter_set[:valid_size, :, :]
          valid_dataset[start_v:end_v, :, :] = valid_letter
          valid_labels = define_labels(label_files, valid_labels, start_v, end_v)
        train_letter = letter_set[valid_size:end_l, :, :]
        train_dataset[start_t:end_t, :, :] = train_letter
        train_labels = define_labels(label_files, train_labels, start_t, end_t, )
    except Exception as e:
      print('Unable to process data from', pickle_file, ':', e)
      raise
    return valid_dataset, valid_labels, train_dataset, train_labels 
            
train_size = 30000
valid_size = 3000
test_size = 13000
# train_size = 18
# valid_size = 2
# test_size = 20

valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(
  train_datasets, train_size, valid_size)
_, _, test_dataset, test_labels = merge_datasets(test_datasets, test_size)

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

1
3
9
Training: (30000, 28, 28, 3) (30000,)
Validation: (3000, 28, 28, 3) (3000,)
Testing: (13000, 28, 28, 3) (13000,)


In [125]:
pickle_file = os.path.join(data_root, 'train.pickle')
try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    }

  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

pickle_file = os.path.join(data_root, 'test.pickle')
try:
  f = open(pickle_file, 'wb')
  save = {
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise
    
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

Compressed pickle size: 310596346
Compressed pickle size: 122356242


In [126]:
pickle_file = 'train.pickle'
with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
    
pickle_file = 'test.pickle'
with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']

  del save  # hint to help gc free up memory
  print('Test set', test_dataset.shape, test_labels.shape)

print(train_labels.shape)


Training set (30000, 28, 28, 3) (30000,)
Validation set (3000, 28, 28, 3) (3000,)
Test set (13000, 28, 28, 3) (13000,)
(30000,)


In [127]:
# image_size = 28
num_labels = 10
num_channels = 3 # 

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
#   labels = tf.one_hot(labels, num_labels)
  return dataset, labels


print(train_labels[1002], train_labels.shape)
print(train_labels[1045], train_labels.shape)
print(train_labels[2456], train_labels.shape)
print(train_labels[12939], train_labels.shape)

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print(train_labels[1002], train_labels.shape)
print(train_labels[1045], train_labels.shape)
print(train_labels[2456], train_labels.shape)
print(train_labels[12939], train_labels.shape)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

3 (30000,)
7 (30000,)
8 (30000,)
6 (30000,)
[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.] (30000, 10)
[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.] (30000, 10)
[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.] (30000, 10)
[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.] (30000, 10)
Training set (30000, 28, 28, 3) (30000, 10)
Validation set (3000, 28, 28, 3) (3000, 10)
Test set (13000, 28, 28, 3) (13000, 10)


In [128]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [133]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
beta = 0.0001
decay_steps = 1000
decay_size = 0.95

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
#   tf_train_labels = tf.placeholder(tf.int32, shape=(batch_size))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  keep_prob = tf.placeholder("float")

  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    maxpool = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(maxpool + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    maxpool = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(maxpool + layer2_biases)
    hidden_layer_drop = tf.nn.dropout(hidden, keep_prob)  
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  print(logits)
  print(tf_train_labels)
  print(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
#   loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
#     logits=logits, labels=tf_train_labels) +
#     beta*tf.nn.l2_loss(layer1_weights) +
#     beta*tf.nn.l2_loss(layer1_biases) +
#     beta*tf.nn.l2_loss(layer4_weights) +
#     beta*tf.nn.l2_loss(layer4_biases)))

  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, decay_steps, decay_size)
  optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
#   train_prediction = logits
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Tensor("add_3:0", shape=(16, 10), dtype=float32)
Tensor("Placeholder_1:0", shape=(16, 10), dtype=float32)
Tensor("Placeholder:0", shape=(16, 28, 28, 3), dtype=float32)


In [134]:
num_steps = 3001

#actual training
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  valor=0
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
#     batch_data = train_dataset[valor:valor+1, :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size)]
    batch_labels = train_labels[offset:(offset + batch_size), :]
#     batch_labels = train_labels[valor:valor+1, :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
#     valor+=1
    if(step>=2000):
      step=0
    if (step % 500 == 0):
#       print(predictions)
#       print(batch_labels)
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.726485
Minibatch accuracy: 6.2%
Validation accuracy: 10.5%
Minibatch loss at step 500: 2.381953
Minibatch accuracy: 0.0%
Validation accuracy: 9.8%
Minibatch loss at step 1000: 2.289570
Minibatch accuracy: 12.5%
Validation accuracy: 9.8%
Minibatch loss at step 1500: 2.340073
Minibatch accuracy: 6.2%
Validation accuracy: 9.8%
Minibatch loss at step 0: 2.301353
Minibatch accuracy: 18.8%


KeyboardInterrupt: 